## Process all the data using TPP

In [33]:
# TODO: Delete this cell before release
# #Change the names of all the fastq files to SRA numbers for testing
# from shutil import copyfile
# import pandas as pd

# samp_df = pd.read_csv('input/Sample_Labels.csv')
# for uid, sra in zip(samp_df['Run_UID'],samp_df['SRA Accession']):
#     f1 = 'input/'+uid + '_1.fastq'
#     f1_out = 'input/'+sra + '_1.fastq'
#     f2 = 'input/'+uid + '_2.fastq'
#     f2_out = 'input/'+sra + '_2.fastq'
    
#     copyfile(f1, f1_out, follow_symlinks=False)
#     copyfile(f2, f2_out, follow_symlinks=False)

In [2]:
%%bash
#SET THESE VARIABLES FOR YOUR LOCAL FILE STRUCTURE:
PATH_TO_DATA=input/fastq #Note: Any fastq files located here will be processed for alignment with tpp
OUT_DIR=output
TPP_OUT_DIR=tpp_out
REFS=input/Genome/GCF_000195955.2_ASM19595v2_genomic.fna
GENBANK_FILE=input/Genome/GCF_000195955.2_ASM19595v2_genomic.gbff

#Definition of variables for processing
PYTHON2=$(which python2)
BWA=$(which bwa)
BWA_ALG="aln"

REPLICON_ID="NC_000962.3"
FASTQ_DIR=$PATH_TO_DATA

PREFIXES_OUTFILE=$OUT_DIR/$TPP_OUT_DIR/`basename $FASTQ_DIR`_prefixes.txt

# These are used for creating a CSV file
CSV_OUTFILE=$OUT_DIR/`basename $FASTQ_DIR`.csv
UNIQUE_FIELDS="locus_tag"
FIELDS="product regulatory_class bound_moiety"

#Parameter settings for tpp
PRIMER=AACCTGTTA
MISMATCHES=2
WINDOW_SIZE=6

###################################################################

#Gunzip all fastq.gz files:
#gunzip -k $FASTQ_DIR/*.fastq.gz

#Process raw fastq files using tpp
COUNTER=0
INITIAL_START_TIME=$SECONDS
for FASTQ in $FASTQ_DIR/*_1.fastq; do
#for FASTQ in $FASTQ_DIR/Rich_6d_H+_1_R1.fastq; do
    echo "******** Run $COUNTER: $FASTQ ********"
    READS1=$FASTQ
    READS2=${FASTQ/_1.fastq/_2.fastq}

    OUTNAME=$(basename $FASTQ)
    OUTNAME=${OUTNAME/_1.fastq/}
    tpp -himar1 -bwa $BWA -bwa-alg $BWA_ALG -ref $REFS -replicon-ids $REPLICON_ID -reads1 $READS1 -reads2 $READS2 \
       -window-size $WINDOW_SIZE -primer $PRIMER -mismatches $MISMATCHES -output $OUT_DIR/$TPP_OUT_DIR/$OUTNAME &
done
wait

echo "Creating prefixes file with all prefixes from all runs..."
basename -a $OUT_DIR/$TPP_OUT_DIR/*.wig | rev | cut -c-4 --complement | rev | uniq > $PREFIXES_OUTFILE
echo "Created '$PREFIXES_OUTFILE'."
echo ""
echo "Creating CSV file with all samples processed by TPP..."
$PYTHON2 scripts/wig_gb_to_csv.py -l $PREFIXES_OUTFILE -g $GENBANK_FILE -u $UNIQUE_FIELDS -f $FIELDS -o $CSV_OUTFILE
echo "Created '$CSV_OUTFILE'."
echo ""
(( TOTAL_RUN_TIME = SECONDS - INITIAL_START_TIME )) 
echo "********** TPP driver script finished in a total of $TOTAL_RUN_TIME seconds **********"

******** Run 0: input/fastq/SRR23906893_1.fastq ********
******** Run 0: input/fastq/SRR23906897_1.fastq ********
# title: Tn-Seq Pre-Processor
# date: 03/31/2023 09:59:12
# command: python /home/will/src_and_bin/Other_Software/miniconda3/envs/tnseq_avium_abx_fordistro/bin/tpp -himar1 -bwa /home/will/src_and_bin/Other_Software/miniconda3/envs/tnseq_avium_abx_fordistro/bin/bwa -bwa-alg aln -ref input/Genome/GCF_000195955.2_ASM19595v2_genomic.fna -replicon-ids NC_000962.3 -reads1 input/fastq/SRR23906893_1.fastq -reads2 input/fastq/SRR23906893_2.fastq -window-size 6 -primer AACCTGTTA -mismatches 2 -output output/tpp_out/SRR23906893
# transposon type: Himar1
# protocol type: Sassetti
# bwa flags:
# read1: input/fastq/SRR23906893_1.fastq
# read2: input/fastq/SRR23906893_2.fastq
# ref_genome: input/Genome/GCF_000195955.2_ASM19595v2_genomic.fna
# replicon_ids: NC_000962.3
# total_reads (or read pairs): 2783134
# trimmed_reads (reads with valid Tn prefix, and insert size>20bp): 2661285
# reads

[tn_preprocess] running pre-processing on input/fastq/SRR23906893_1.fastq
[tn_preprocess] protocol: Sassetti
[tn_preprocess] transposon type: Himar1
[tn_preprocess] running pre-processing on input/fastq/SRR23906897_1.fastq
[tn_preprocess] protocol: Sassetti
[tn_preprocess] transposon type: Himar1
[tn_preprocess] One reference genome specified: input/Genome/GCF_000195955.2_ASM19595v2_genomic.fna, containing 1 records.
[tn_preprocess] extracting reads...
[tn_preprocess] fastq2reads: input/fastq/SRR23906893_1.fastq -> output/tpp_out/SRR23906893.reads1
[tn_preprocess] One reference genome specified: input/Genome/GCF_000195955.2_ASM19595v2_genomic.fna, containing 1 records.
[tn_preprocess] extracting reads...
[tn_preprocess] fastq2reads: input/fastq/SRR23906897_1.fastq -> output/tpp_out/SRR23906897.reads1
[tn_preprocess] 1000000 reads processed
[tn_preprocess] 1000000 reads processed
[tn_preprocess] 2000000 reads processed
[tn_preprocess] 2000000 reads processed
[tn_preprocess] 3000000 read

In [18]:
%%bash
#Run analysis script (depends on the file output/input.csv in order to run)

python3 scripts/hypersus_analysis.py 

JT test run time: 2.134294033050537
JT test run time: 134.73224663734436
JT test run time: 1.8231902122497559
JT test run time: 133.4129159450531


/home/will/Professional/JHU/Research/baderlab/Drafts/2022-08-06_Mtb_Hypersusceptibility_paper/final_code/PBS_analysis/scripts/hypersus_helper.py:61: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name,g in df.groupby(['uid'],sort=False):
/home/will/Professional/JHU/Research/baderlab/Drafts/2022-08-06_Mtb_Hypersusceptibility_paper/final_code/PBS_analysis/scripts/hypersus_helper.py:61: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name,g in df.groupby(['uid'],sort=False):
/home/will/Professional/JHU/Research/baderlab/Drafts/2022-08-06_Mtb_Hypersusceptibility_paper/final_code/PBS_analysis/scripts/hypersus_helper.py:61: FutureWarning: In a

In [19]:
#Reorganize data for TableS1
infold = 'output/hypersus_analysis/'
outfold = 'output/hypersus_analysis/'

lfc_thresh = 0.5
padj_thresh = 0.05

import pandas as pd
import numpy as np
from math import log10, floor

def round_sig(x, sig=2):
    #Implement significant figures in python
    if np.isnan(x):
        return(np.nan)
    return round(x, sig-int(floor(log10(abs(x))))-1)

def make_tables1(infilename12h, infilename48h, outfilename, treatment, drug_dose_ls):
    #Merges tables output from analysis and changes formatting for disribution (TableS1)
    
    abx12_df = pd.read_csv(infilename12h).rename({'pval-adj (BH)':'padj'},axis='columns')
    abx48_df = pd.read_csv(infilename48h).rename({'pval-adj (BH)':'padj'},axis='columns')

    abx12_df[treatment+['pval','padj']] = abx12_df[treatment+['pval','padj']].applymap(round_sig)
    abx48_df[treatment+['pval','padj']] = abx48_df[treatment+['pval','padj']].applymap(round_sig)

    merged_df = abx12_df.merge(abx48_df, how='inner',on=['uid','product'],suffixes=('_7d','_14d'))
    merged_df['Prediction'] = 'Not Significant'
    hypersus_bool = ((merged_df['padj_7d'] < padj_thresh) & \
                (merged_df[treatment[-1]+'_7d'] < -lfc_thresh)) & \
                ((merged_df['padj_14d'] < padj_thresh) & \
                (merged_df[treatment[-1]+'_14d'] < -lfc_thresh))
    hypertol_bool = ((merged_df['padj_7d'] < padj_thresh) & \
                (merged_df[treatment[-1]+'_7d'] > lfc_thresh)) & \
                ((merged_df['padj_14d'] < padj_thresh) & \
                (merged_df[treatment[-1]+'_14d'] > lfc_thresh))

    if (len(treatment) == 1):
        merged_df = merged_df.rename({treatment[0]+'_7d':drug_dose_ls[0]+' - LFC_7d', \
            treatment[0]+'_14d':drug_dose_ls[0]+' - LFC_14d'}, axis=1)

    elif (len(treatment) ==2):
        merged_df = merged_df.rename({treatment[0]+'_7d':drug_dose_ls[0]+' - LFC_7d', \
            treatment[1]+'_7d':drug_dose_ls[1]+' - LFC_7d', \
            treatment[0]+'_14d':drug_dose_ls[0]+' - LFC_14d', \
            treatment[1]+'_14d':drug_dose_ls[1]+' - LFC_14d'}, axis=1).drop(['R+_7d','R+_14d'],axis=1)

    else:
        exit("Number of doses should be either 1 or 2")
    merged_df = merged_df.rename({'uid':'Genomic Feature','product':'Genbank Annotation'}, axis=1)

    merged_df['Prediction'][hypersus_bool]='Hypersusceptible'
    merged_df['Prediction'][hypertol_bool]='Hypertolerant'

    merged_df.to_csv(outfilename,index=False)
    
treatment = ['R++','R+++']
drug_dose_ls = ['Rifampin (0.4ug/mL)','Rifampin (4.0ug/mL)']
make_tables1(infold+'summary_7d_RMP.csv', infold+'summary_14d_RMP.csv', outfold+'TableS3_RMP.csv', treatment, drug_dose_ls)
    
treatment = ['H+++']
drug_dose_ls = ['Isoniazid (1.0ug/mL)']
make_tables1(infold+'summary_7d_INH.csv', infold+'summary_14d_INH.csv', outfold+'TableS4_INH.csv', treatment, drug_dose_ls)

/home/will/src_and_bin/Other_Software/miniconda3/envs/tnseq_avium_abx_fordistro/lib/python2.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/will/src_and_bin/Other_Software/miniconda3/envs/tnseq_avium_abx_fordistro/lib/python2.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
